In [7]:
from astropy.io import fits
from reproject import reproject_interp
import warnings
warnings.filterwarnings('ignore')

In [8]:
datadir='/Users/atetarenko/Desktop/Research/roberto_GC/fits_ims/'
fits_file_org=datadir+'WISE-Band-311.fits'
fits_file_proj=datadir+'v1.0.2_super_gc_13pca_map50_crop_cutout_1594.fits'
fits_out=datadir+'v1.0.2_super_gc_13pca_map50_crop_cutout_1594_regrid2.fits'

hdu1 = fits.open(fits_file_org)[0]
hdu2 = fits.open(fits_file_proj)[0]

array, footprint = reproject_interp(hdu2, hdu1.header)
fits.writeto(fits_out, array, hdu1.header, clobber=True)